In [4]:
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression as LR

In [6]:
treatment = [] #X
outcomes = [] #Y
features = [] #Z

with open('ihdp.csv') as csvDataFile:
    csvReader = csv.reader(csvDataFile)
    next(csvReader)
    for x in range(2, 27):
        features.append([])
    for row in csvReader:
        treatment.append(row[0])
        outcomes.append(row[1])
        for i in range(0, 25):
            features[i].append(row[i+2])

In [7]:
lr = LR(penalty='none', max_iter=200, random_state=0, solver='lbfgs')
lr.fit(np.array(features).transpose(), treatment)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=200,
                   multi_class='warn', n_jobs=None, penalty='none',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [8]:
def estimate_treatment_effect(model, X, Y, Z):
    score = model.predict_proba(Z.transpose())
    e_z = score[:,1]
    result = 0
    for i in range(0, 747):
        result += (X[i] - e_z[i]) * Y[i] / (e_z[i] * (1 - e_z[i]))
    return result / 747

In [9]:
X = (np.array(treatment)).astype(float)
Y = (np.array(outcomes)).astype(float)
Z = np.array(features).astype(float)
estimate = estimate_treatment_effect(lr, X, Y, Z)
print(estimate)

3.6956988430118836


In [10]:
count1 = 0
total1 = 0
count2 = 0
total2 = 0
for i in range(0, 747):
    if X[i] == 0:
        count1 += Y[i]
        total1 += 1
    if X[i] == 1:
        count2 += Y[i]
        total2 += 1
print(count2 / total2 - count1 / total1)

4.021121012430834
